In [261]:
import xarray as xr
import numpy as np
import cftime
import cartopy.crs as ccrs
from scipy.ndimage import gaussian_filter
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd

In [262]:
def manipulation_data():
    """
    Função para manipular e salvar o netcdf
    """
    try:
        # Caminho dos arquivos
        arquivos_nc = [
            "data/pr_Amon_HadGEM3-GC31-MM_historical_r4i1p1f3_gn_185001-186912.nc",
            "data/pr_Amon_HadGEM3-GC31-MM_historical_r4i1p1f3_gn_187001-188912.nc",
            "data/pr_Amon_HadGEM3-GC31-MM_historical_r4i1p1f3_gn_189001-190912.nc",
            "data/pr_Amon_HadGEM3-GC31-MM_historical_r4i1p1f3_gn_191001-192912.nc",
            "data/pr_Amon_HadGEM3-GC31-MM_historical_r4i1p1f3_gn_193001-194912.nc",
            "data/pr_Amon_HadGEM3-GC31-MM_historical_r4i1p1f3_gn_195001-196912.nc",
            "data/pr_Amon_HadGEM3-GC31-MM_historical_r4i1p1f3_gn_197001-198712.nc",
            "data/pr_Amon_HadGEM3-GC31-MM_historical_r4i1p1f3_gn_198801-198912.nc",
            "data/pr_Amon_HadGEM3-GC31-MM_historical_r4i1p1f3_gn_199001-200912.nc",
            "data/pr_Amon_HadGEM3-GC31-MM_historical_r4i1p1f3_gn_201001-201412.nc",
        ]

        # Abrir e concatenar os arquivos
        data = xr.open_mfdataset(arquivos_nc, combine='by_coords', engine='netcdf4')

        # Salvar o arquivo
        data.to_netcdf("precipitacao_hadgem3_bacia_sf.nc")
        print("Arquivo salvo com sucesso!")

        # Abrir o arquivo novo
        ds = xr.open_dataset("precipitacao_hadgem3_bacia_sf.nc")

        # Remover variáveis desnecessárias
        ds = ds.drop_vars(["lat_bnds", "lon_bnds", "time_bnds"])

        # Recortando o período de 1980 a 2010
        ds = ds.sel(time=slice("1980-01-01", "2010-12-30"))

        # Filtrar latitudes do Brasil
        ds = ds.sel(lat=slice(-35, 5))

        # Ajustar longitudes, convertendo de 0-360 para -180 a 180
        if ds.lon.max() > 180:
            ds = ds.assign_coords(lon=(ds.lon - 360))
        
        # Ordenar longitudes
        ds = ds.sortby("lon")
        
        # Filtrar longitudes do Brasil
        ds = ds.sel(lon=slice(-74, -32))
        
        # Converter a precipitação de kg/m²/s para mm/dia
        ds["pr"] = ds["pr"] * 86400
        
        # Salvar o novo NetCDF filtrado e convertido
        ds.to_netcdf("precipitacao_hadgem3_brasil_1980_2010_convertido.nc")
        print("Arquivo salvo com sucesso!")
    except Exception as e:
        print(f"Erro: {e}")

In [263]:
def load_shape() -> gpd.GeoDataFrame:
    """
    Função para carregar o shapefile

    Return:
        gdf (GeoDataFrame): Shapefile carregado
    """
    try:
        # Caminho do Shapefile
        shapefile_path = "data/shape/Nivel02_PropostoPolygon.shp"

        #Lê o shapefile
        gdf = gpd.read_file(shapefile_path)

        return gdf
    except FileNotFoundError:
        print("Arquivo não encontrado!")
    except Exception as e:
        print(f"Erro: {e}")
        return None

In [264]:
def load_netcdf(path: str) -> xr.Dataset:
    """
    Função para carregar o NetCDF

    Args:
        path (str): Caminho do arquivo NetCDF

    Return:
        ds (Dataset): NetCDF carregado
    """
    try:
        # Carregar o NetCDF
        ds = xr.open_dataset(path)

        return ds
    except FileNotFoundError:
        print("Arquivo não encontrado!")
    except Exception as e:
        print(f"Erro: {e}")
        return None

In [265]:
def extract_years(data):
    """
    Extrai os anos da variável 'time' do NetCDF.
    """
    try:
        time_var = data["time"]
        years = np.array([t.year for t in time_var.values])
        return years
    except Exception as e:
        print(f"Erro ao extrair anos: {e}")
        return None

In [266]:
def manipulation_shape(netcdf: xr.DataArray, shapefile: gpd.GeoDataFrame):
    """
    Função para manipular o shapefile

    Args:
        netcdf (DataArray): NetCDF carregado
        shapefile (GeoDataFrame): Shapefile carregado
    """
    try:
        # Região do Shape
        xmin, ymin, xmax, ymax = shapefile.total_bounds

        # Filtrando no NetCDF
        netcdf = netcdf.sel(
            lat=slice(ymin, ymax),
            lon=slice(xmin, xmax)
        )
    except Exception as e:
        print(f"Erro: {e}")

In [267]:
def calc_annual(dataset: xr.DataArray) -> xr.DataArray:
    """
    Função para calcular a soma anual

    Args:
        dataset (DataArray): Dataset com os dados de precipitação

    Return:
        xr.DataArray: Dataset calculado anualmente
    """
    try:
        # Calcula ano a ano
        data_annual = dataset['pr'].groupby('time.year').sum('time').compute()

        return data_annual
    except Exception as e:
        print(f"Erro: {e}")
        return None

In [268]:
def plot_map_from_netcdf(data_annual: xr.DataArray, title: str, ax: plt.Axes, shape: gpd.GeoDataFrame) -> plt.Axes:
    """
    Função para plotar os dados de precipitação anual de um DataArray já calculado e retornar a imagem gerada.
    
    Args:
        data_annual (xr.DataArray): DataArray contendo os dados de precipitação anual (já calculados)
        title (str): Título do gráfico
        ax (plt.Axes): Eixo do gráfico onde a imagem será plotada
        shape (gpd.GeoDataFrame): Shape da região de interesse (ex: bacia do Rio São Francisco)
    
    Returns:
        plt.Axes: Eixo com a imagem plotada
    """
    try:
        # Se você quiser plotar um ano específico (ex: 1980)
        # Seleciona os dados para o ano 1980
        # data_for_year = data_annual.sel(year=1980)

        # Ou, se quiser fazer a média de todos os anos
        data_for_year = data_annual.mean('year')

        # Aumentando a resolução dos dados por interpolação
        data_high_res = data_for_year.interp(lat=np.linspace(data_for_year.lat.min(), data_for_year.lat.max(), 1000),
                                             lon=np.linspace(data_for_year.lon.min(), data_for_year.lon.max(), 1000),
                                             method='linear')

        # Suavização dos dados
        smoothed_data = gaussian_filter(data_high_res.values, sigma=0.5)
        data_high_res = xr.DataArray(smoothed_data, coords=data_high_res.coords, dims=data_high_res.dims)

        # Plotando os dados de precipitação anual
        im = data_high_res.plot.pcolormesh(
            ax=ax, cmap='Blues', transform=ccrs.PlateCarree(), add_colorbar=False, shading='auto'
        )

        # Adiciona título ao gráfico
        ax.set_title(title)

        # Definindo os limites automaticamente com base no shapefile
        shape_bounds = shape.bounds
        lon_min, lon_max = shape_bounds['minx'].min(), shape_bounds['maxx'].max()
        lat_min, lat_max = shape_bounds['miny'].min(), shape_bounds['maxy'].max()

        ax.set_xlim([lon_min, lon_max])
        ax.set_ylim([lat_min, lat_max])

        # Marcando os limites do shape da bacia no mapa
        shape.boundary.plot(ax=ax, edgecolor='black', linewidth=1)

        # Adiciona a legenda de cores (escala de precipitação)
        cbar = plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.02, pad=0.04)
        cbar.set_label('Precipitação (mm)', fontsize=12)

        # Configurações de ticks para longitude e latitude com base no shapefile
        ax.set_xticks(np.arange(lon_min, lon_max, (lon_max - lon_min) / 5), crs=ccrs.PlateCarree())
        ax.set_yticks(np.arange(lat_min, lat_max, (lat_max - lat_min) / 5), crs=ccrs.PlateCarree())
        # Formatação do Gráfico
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')

        # Retorna o eixo com a imagem gerada
        return ax

    except Exception as e:
        print(f"Erro ao gerar o mapa: {e}")
        return None


In [269]:
def plot_graphic_sum(data: xr.DataArray, title: str, ax: plt.Axes) -> plt.Axes:
    """
    Função para preparar um gráfico de linha mostrando a soma total da precipitação por ano.

    Args:
        data (xr.DataArray): DataArray contendo os dados de precipitação.
        title (str): Título do gráfico.
        ax (plt.Axes): Eixo onde o gráfico será plotado.

    Returns:
        plt.Axes: O eixo com o gráfico preparado.
    """
    try:
        # Calcula a soma total de precipitação por ano
        sum_per_year = data['pr'].groupby('time.year').sum(dim='time').sum(dim=('lat', 'lon'))

        # Plotando a linha
        ax.plot(sum_per_year['year'], sum_per_year, marker='o', linestyle='-', color='b', label="Precipitação Total")

        # Configurações do gráfico
        ax.set_title(title, fontsize=14)
        ax.set_xlabel("Ano", fontsize=12)
        ax.set_ylabel("Precipitação Total (mm)", fontsize=12)
        ax.grid(True, linestyle='--', alpha=0.6)
        ax.legend()

        return ax

    except Exception as e:
        print(f"Erro ao preparar o gráfico de soma da precipitação: {e}")
        return None


In [270]:
def configure_layout():
    """
    Configura o layout para exibir um mapa e um gráfico de linha.
    
    Returns:
        tuple: (fig, ax_map, ax_graph) - Figura principal e os eixos do mapa e do gráfico.
    """
    try:
        # Criar a figura com dois subplots (mapa e gráfico)
        fig = plt.figure(figsize=(16, 7), dpi=300)
        gs = gridspec.GridSpec(1, 2, width_ratios=[2, 1])

        # Criar os subplots
        ax_map = plt.subplot(gs[0], projection=ccrs.PlateCarree())
        ax_graph = plt.subplot(gs[1])

        # Ajustar espaçamento entre os gráficos
        plt.subplots_adjust(left=0.05, right=0.95, top=0.9, bottom=0.1, wspace=0.3)

        return fig, ax_map, ax_graph

    except Exception as e:
        print(f'Erro ao configurar o layout: {e}')
        return None, None, None


In [271]:
def main():
    """
    Função principal
    """
    try:
        # Carregar o shapefile
        shape = load_shape()

        # Carregar o NetCDF
        data = load_netcdf("precipitacao_hadgem3_bacia_sf.nc")

        # Verificando o tipo de 'time'
        time_values = data['time'].values
        print(type(time_values[0]))  # Verificando o tipo da data (deve ser cftime)

        # Convertendo as datas para o formato datetime usando cftime
        if isinstance(time_values[0], cftime._cftime.Datetime360Day):
            time_dates = [pd.to_datetime(cftime.num2date(t, data['time'].units)) for t in time_values]
        else:
            time_dates = pd.to_datetime(time_values)

        # Exibindo as primeiras datas
        print(time_dates[:10])  # Mostrar as 10 primeiras datas

        # Manipular o shapefile
        manipulation_shape(data, shape)

        # Calcular a média anual
        data_annual = calc_annual(data)
        
        fig, ax1, ax2 = configure_layout()

        # Configurar o primeiro subplot (Mapa)
        plot_map_from_netcdf(data_annual, "Precipitação Anual Total", ax1, shape)

        # Configurar o segundo subplot (Gráfico de linha)
        plot_graphic_sum(data, "Soma da Precipitação Anual", ax2)

        # Ajustar layout e exibir o gráfico
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Erro durante a execução: {e}")

if __name__ == "__main__":
    main()


<class 'cftime._cftime.Datetime360Day'>
Erro durante a execução: 'DataArray' object has no attribute 'units'
